# Convolutional Neural Network in TensorFlow


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
data1 = pd.read_csv('./Nor96.csv', sep=",")
data2 = pd.read_csv('./Ab96.csv', sep=",")

data=data1.append(data2,ignore_index = True)

In [2]:
X = data[data.columns[0:27648]]

In [3]:
X = X.as_matrix().astype(np.float32)
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [4]:
y = data[data.columns[27648:27650]]
y = y.as_matrix().astype(np.float32)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=200)

In [6]:
import tensorflow as tf

In [7]:
# Parameters
# learning_rate = 0.001
learning_rate = 0.005
training_iters = 100000
batch_size = 21
display_step = 20

In [8]:
# Network Parameters
n_input = 27648 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-2 digits)
dropout = 0.75 # Dropout, probability to keep units

In [9]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [10]:
# Create model
def conv2d(img, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img, w, strides=[1, 1, 1, 1], 
                                                  padding='SAME'),b))

def max_pool(img, k):
    return tf.nn.max_pool(img, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(_X, _weights, _biases, _dropout):
    # Reshape input picture
    _X = tf.reshape(_X, shape=[-1, 96, 96, 3])

    # Convolution Layer
    conv1 = conv2d(_X, _weights['wc1'], _biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = max_pool(conv1, k=2)
    # Apply Dropout
    conv1 = tf.nn.dropout(conv1, _dropout)

    # Convolution Layer
    conv2 = conv2d(conv1, _weights['wc2'], _biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = max_pool(conv2, k=2)
    # Apply Dropout
    conv2 = tf.nn.dropout(conv2, _dropout)
    
    # Convolution Layer
    conv3 = conv2d(conv2, _weights['wc3'], _biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = max_pool(conv3, k=2)
    # Apply Dropout
    conv3 = tf.nn.dropout(conv3, _dropout)
    
    # Convolution Layer
    conv4 = conv2d(conv3, _weights['wc4'], _biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = max_pool(conv4, k=2)
    # Apply Dropout
    conv4 = tf.nn.dropout(conv4, _dropout)

    # Convolution Layer
    conv5 = conv2d(conv4, _weights['wc5'], _biases['bc5'])
    # Max Pooling (down-sampling)
    conv5 = max_pool(conv5, k=2)
    # Apply Dropout
    conv5 = tf.nn.dropout(conv5, _dropout)

    # Fully connected layer
    # Reshape conv2 output to fit dense layer input
    dense1 = tf.reshape(conv5, [-1, _weights['wd1'].get_shape().as_list()[0]]) 
    # Relu activation
    dense1 = tf.nn.relu(tf.add(tf.matmul(dense1, _weights['wd1']), _biases['bd1']))
    # Apply Dropout
    dense1 = tf.nn.dropout(dense1, _dropout) # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(dense1, _weights['out']), _biases['out'])
    return out

In [11]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 3, 96])), 
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 96, 192])), 
    'wc3': tf.Variable(tf.random_normal([5, 5, 192, 384])), 
    'wc4': tf.Variable(tf.random_normal([5, 5, 384, 768])), 
    'wc5': tf.Variable(tf.random_normal([5, 5, 768, 1536])), 
    

    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([3*3*1536, 3072])), 
    # 1024 inputs, 2 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([3072, n_classes])) 
}

biases = {
    'bc1': tf.Variable(tf.random_normal([96])),
    'bc2': tf.Variable(tf.random_normal([192])),
    'bc3': tf.Variable(tf.random_normal([384])),
    'bc4': tf.Variable(tf.random_normal([768])),
    'bc5': tf.Variable(tf.random_normal([1536])),
    'bd1': tf.Variable(tf.random_normal([3072])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [12]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

In [13]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [14]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
# Initializing the variables
init = tf.initialize_all_variables()

In [16]:
def batch_gen(X, y, N):
    while True:
        idx = np.random.choice(len(y), N)
        yield X[idx].astype('float32'), y[idx].astype('float32')

In [17]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:        
        train_batches = batch_gen(x_train, y_train, batch_size)
        batch_xs, batch_ys = next(train_batches)
        # Fit training using batch data
        sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch accuracy
            acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            # Calculate batch loss
            loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
            print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                "{:.5f}".format(acc))
            print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: x_test, 
                                                             y: y_test, 
                                                             keep_prob: 1.}))
        step += 1

Iter 420, Minibatch Loss= 90782162944.000000, Training Accuracy= 0.47619
('Testing Accuracy:', 0.52380955)
Iter 840, Minibatch Loss= 9079764992.000000, Training Accuracy= 0.57143
('Testing Accuracy:', 0.54761904)
Iter 1260, Minibatch Loss= 5027045888.000000, Training Accuracy= 0.61905
('Testing Accuracy:', 0.66666669)
Iter 1680, Minibatch Loss= 6387364352.000000, Training Accuracy= 0.61905
('Testing Accuracy:', 0.54761904)
Iter 2100, Minibatch Loss= 1341291648.000000, Training Accuracy= 0.95238
('Testing Accuracy:', 0.66666669)
Iter 2520, Minibatch Loss= 1008756608.000000, Training Accuracy= 0.85714
('Testing Accuracy:', 0.71428573)
Iter 2940, Minibatch Loss= 1542975872.000000, Training Accuracy= 0.71429
('Testing Accuracy:', 0.78571427)
Iter 3360, Minibatch Loss= 4526496256.000000, Training Accuracy= 0.52381
('Testing Accuracy:', 0.73809522)
Iter 3780, Minibatch Loss= 679201024.000000, Training Accuracy= 0.90476
('Testing Accuracy:', 0.64285713)
Iter 4200, Minibatch Loss= 9265269760.0

KeyboardInterrupt: 